In [1]:
from data_read import *
from odometry_functions import *
import matplotlib.pyplot as plt
import numpy as np
import cv2
from scipy.optimize import least_squares
import os

In [2]:
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.2,
                       minDistance = 7,
                       blockSize = 7,
                       useHarrisDetector=False)
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

In [3]:
data_loader=Data_load(input_line=64, frames=5)

In [4]:
lidar_frames,gt_frames,img_frames,index_list=data_loader.read_frames(batch=1, if_removal=False)
print(index_list)

[84917]


In [5]:
features = find_keypoints(img_frames[0,:,:,:],lidar_frames[0,:,:,0],feature_params)
next_features, status, error = track_features(img_frames[0,:,:,:],features,img_frames[1,:,:,:],lk_params)
print(features)

[[[ 736.  180.]]

 [[ 650.  243.]]

 [[1077.  308.]]

 [[ 755.  244.]]]


In [6]:
features = features[status==1]
next_features = next_features[status==1]

In [7]:
scene=data_loader.img_path[index_list[0]].split('/')[5].split('_')[:3]
scene_date = scene[0]
camera_id = data_loader.img_path[index_list[0]].split('/')[-3]
for i in range(1,len(scene)) : 
    scene_date += '_' + scene[i]
K = get_calib_file(scene_date,camera_id) # eg: 2011_09_26, image_03
# print(K)
pts_3d_1 = compute_3D_points(lidar_frames[0,:,:,:],features,K)
pts_2d_2 = []
for i in range(len(features)):
    x = int(next_features[i,0])
    y = int(next_features[i,1])
    pts_2d_2.append([x,y,1])
pts_2d_2 = np.asarray(pts_2d_2)
print(pts_3d_1)
print(len(pts_2d_2))
print(pts_2d_2)
# print(features)
P = np.hstack((K,np.zeros((3,1))))


[[ 1.3174588  -1.94119225 29.63671875  1.        ]
 [-0.69915641  0.06168356 13.82421875  1.        ]
 [ 2.45017307  0.44579587  5.81640625  1.        ]
 [ 0.8904539   0.07581639 13.60546875  1.        ]]
4
[[ 750  181    1]
 [ 644  237    1]
 [1071  283    1]
 [ 792  261    1]]


In [8]:
opt = least_squares(func,np.zeros(6),method='lm',verbose=1,args=(pts_3d_1,pts_2d_2,P))

The maximum number of function evaluations is exceeded.
Function evaluations 4200, initial cost 3.5842e+06, final cost 1.5512e+06, first-order optimality 1.51e+06.


In [9]:
R = genEulerZXZMatrix(opt.x[0],opt.x[1],opt.x[2])
t = np.asarray([[opt.x[3]],[opt.x[4]],[opt.x[5]]])
T = np.hstack((R,t))
T = np.vstack((T,np.asarray([[0,0,0,1]])))
print(T)

[[ 0.99960787  0.02795811 -0.00156348 -0.1526515 ]
 [-0.02798175  0.99944631 -0.01800286  0.28032621]
 [ 0.00105929  0.01803955  0.99983671 -0.38164533]
 [ 0.          0.          0.          1.        ]]
